# Facial Keypoint Detection


## Initialize Environment

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
#from tensorflow.keras.layers import LeakyReLU, ReLU
#from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import Activation, Convolution2D, BatchNormalization, Flatten
#from tensorflow.keras.layers import Dense, Dropout, Input, MaxPool2D, Layer, Reshape, Lambda
#from tensorflow.keras.optimizers import Nadam
#from keras import backend

import keras
from keras.models import Model
from keras.layers import Conv2D, Dense, Input, Reshape, Lambda, Layer, Flatten
from keras.layers import LeakyReLU, BatchNormalization, MaxPool2D
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow 
from keras import initializers
from keras.utils import to_categorical
from keras.layers.core import Activation
import pandas as pd
from keras.optimizers import Nadam
from keras import backend as K
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import cv2
import os, gc, json, math
from keras import initializers
from keras.utils import to_categorical
from keras.layers.core import Activation

## Functions

In [7]:
def convert_pixels(data):
    """
    Convert pixels to the right intensity 0-1 and in a square matrix.
    """
    data = np.array([row.split(' ') for row in data['Image']],dtype='float') / 255.0
    data = data.reshape(-1,96,96,1)
    return(data)

In [8]:
def view_img(sample_img,coord=None):
    """
    Display an image. For debugging, display a coordinate on the image.
    input:
        - sample_img: numpy array. image to be displayed
        - coord: lst. of coordinates in form [[x_coordinate,y_coordinate],[x_coordinate,y_coordinate]]
    TODO handle multiple coordinates. Work out bugs with multiple coordinates
    """
    plt.figure()
    plt.imshow(sample_img.reshape(96,96),cmap='gray')
    if coord is not None:
        plt.scatter(coord[0],coord[1],marker = '*',c='r')
    plt.show()

In [9]:
def get_facial_keypoints(data,ind):
    """
    Structure the coordinates for all facial keypoints for a single image.
    inputs:
        - data: numpy array containing rows as each image sample and columns as facial keypoint coordinates
        - ind: index of the image
    output:
        - numpy array with format [[list of x-coordinates],[list of y-coordinates]]
    """
    data[ind]
    it = iter(data[ind])
    x_coord = []
    y_coord = []

    for x in it:
        x_coord.append(x)
        y_coord.append(next(it))
    
    return(np.array([x_coord,y_coord]))


## Import Data

In [10]:
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    
reset_keras()    

100


In [11]:
if not os.path.exists('../output/'):
    os.makedirs('../output/model')
    os.makedirs('../output/history')
    
    
model_dir = "../output/model/"
history_dir = "../output/history/"

train_file = '../input/training/training.csv'
test_file = '../input/test/test.csv'
train_data = pd.read_csv(train_file)  
#test_data = pd.read_csv(test_file)


bad_samples = [1747, 1731, 1877, 1881, 1979, 2199, 2289, 2321, 2453, 3173, 3296, 3447, 4180, 6859,
              2090, 2175, 1907, 2562, 2818, 3296, 3447, 4263, 4482, 4490, 4636, 5059, 6493, 6585, 6906]

train_data = train_data.drop(bad_samples).reset_index(drop=True)
train_clean = train_data.dropna()


Create training vector with images and normalize thee

In [12]:
x_train = convert_pixels(train_data)

y_train = train_data[[col for col in train_data.columns if col != 'Image']].to_numpy()


Generate labels 

In [13]:

imputer = IterativeImputer(max_iter=1000, tol=0.01, random_state=1)
y_train = imputer.fit_transform(y_train)

#imputer = IterativeImputer(max_iter=1000, tol=0.01, random_state=2)
#y_clean = imputer.fit_transform(y_clean)

bad_bottom_lip = [210, 350, 499, 512, 810, 839, 895, 1058, 1194,1230, 1245, 1546, 1548]
for sample in bad_bottom_lip:
    y_train[sample][29] = 94
    y_train[sample][28] = y_train[sample][26]
   

## Feature Engineering

### Set feature engineering parameters

In [14]:
fill_na = False
add_flip_horiz = True
add_blur_img = False
add_rotate_img = False
add_contrast_img = False
add_translate_img = False
orig_x_train = x_train.copy()
orig_y_train = y_train.copy()


### Fill NA in the training labels.

In [15]:
if fill_na:
    # https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns
    # get column means
    col_mean = np.nanmean(y_train,axis=0)

    # find the x,y indices that are missing from y_train
    inds = np.where(np.isnan(y_train))

    # fill in missing values in y_train with the column means. "take" is much more efficient than fancy indexing
    y_train[inds] = np.take(col_mean, inds[1])


### Flip images horizontally and add to the training data

In [16]:
def flip_img_horiz(train_data):
    """
    Flip images horizontally for all training images
    """
    # Flip images
    x_train = convert_pixels(train_data)
    flip_img = np.array([np.fliplr(x_train[[ind]][0]) for ind in range(x_train.shape[0])])
    
    # Flip coordinates
    train_data_flip = train_data.copy()
    x_columns = [col for col in train_data.columns if '_x' in col]
    train_data_flip[x_columns] = train_data[x_columns].applymap(lambda x: 96-x)
    
    #left and right are swapped so undo
    left_columns = [col for col in train_data.columns if 'left' in col]
    right_columns = [col for col in train_data.columns if 'right' in col]
    train_data_flip[left_columns+right_columns] = train_data_flip[right_columns+left_columns]
    
    flip_coord = train_data_flip[[col for col in train_data if col != 'Image']].to_numpy()
    return(flip_img,flip_coord)

if add_flip_horiz:
    # Apply the augmentation and add the new data to the training set
    flipped_img,flipped_coord = flip_img_horiz(train_clean)
    
   
    x_train = np.append(x_train,flipped_img,axis=0)
    y_train = np.append(y_train,flipped_coord,axis=0)
    
    

### Add Gaussian blurring with a 5x5 filter with $\sigma$ = 2.

In [17]:
def blur_img():
    """
    Add Gaussian blurring to the images
    """
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
    blur_img = np.array([cv2.GaussianBlur(orig_x_train[[ind]][0],(5,5),2).reshape(96,96,1) for ind in range(orig_x_train.shape[0])])
    
    return(blur_img)

if add_blur_img:
    x_train = np.append(x_train,blur_img(),axis=0)
    y_train = np.append(y_train,orig_y_train,axis=0)

### Add image rotation

In [18]:
def rotate_img(x_train, y_train):
    """"
    Rotate images by angles between [5, 10, 14 degrees]
    """
    angles = [5, -5, 10, -10, 14, -14]
    b = np.ones((1,3))
    rows,cols = (96,96)
    x_train_rot = []
    y_train_rot = y_train.copy()
    M_angles = [cv2.getRotationMatrix2D((cols/2,rows/2),angle,1) for angle in angles]
    
    for i in range(x_train.shape[0]):
        #M = cv2.getRotationMatrix2D((cols/2,rows/2),np.random.choice(angles,1),1)
        M = M_angles[np.random.choice(len(M_angles))]
        x_train_rot.append((cv2.warpAffine(x_train[[i]].reshape(rows,cols,1),M,(cols,rows)).reshape(96,96,1)))
       
        #apply affine transformation to (x,y) labels
        for j in range(int(y_train.shape[1]/2)):
            b[:,0:2] = y_train[i,2*j:2*j+2]
            y_train_rot[i,2*j:2*j+2] = np.dot(b,M.transpose()) 
    
    x_train_rot = np.array(x_train_rot)
    return x_train_rot, y_train_rot

if add_rotate_img:
    
    x_rotate, y_rotate = rotate_img(x_train,y_train)
    x_train = np.append(x_train,x_rotate,axis=0)
    y_train = np.append(y_train,y_rotate,axis=0)   
    

### Add image contrast

In [19]:
def contrast_img(apply_random,brightness=.5):
    """
    Add brighter and darker images to the training set with the range of pixel values allowed set at 0 and 1
    Only applies contrast to the original images. Need to make sure we're not setting random seed
    input:
        - brightness: float between -1 and 1
        - apply_random: boolean. Applies random brightness to every sample in the data set.
          Ignores brightness setting
    """
    if apply_random:
        # uses a .1 increment [-.3,.8) to pick out a brightness number for each sample
        # numbers are chosen such that they are still realistically visible and
        # the added training data has an appreciable change in contrast
        brightness = np.random.choice(np.round(np.arange(-.3,.8,.1),2),size=orig_x_train.shape[0])
        bright_img = np.array([orig_x_train[[ind]][0]+brightness[ind] for ind in range(orig_x_train.shape[0])])
    else:
        bright_img = orig_x_train + brightness

    bright_img[bright_img > 1] = 1
    bright_img[bright_img < 0] = 0
    return(bright_img)
    
if add_contrast_img:
    # create two sets of images undergoing contrast changes
    x_train = np.append(x_train,contrast_img(apply_random=False),axis=0)
    y_train = np.append(y_train,orig_y_train,axis=0)
    
# testing code
# view_img(contrast_img(apply_random=True)[[30]])
# view_img(contrast_img(apply_random=True)[[30]])
# view_img(contrast_img(apply_random=True)[[30]])


In [20]:
def translate_img():
    """
    Add translational shift to the images randomly
    """
    trans_train_data = train_data.copy()
    
    rows,cols = (96,96)
    shift_x = 96*np.random.choice(np.arange(-.4,.4,.05),size=trans_train_data.shape[0])
    shift_y = 96*np.random.choice(np.arange(-.4,.4,.05),size=trans_train_data.shape[0])
    M = [np.float32([[1,0,shift_x[ind]],[0,1,shift_y[ind]]]) for ind in range(trans_train_data.shape[0])]
    trans_img = np.array([cv2.warpAffine(x_train[[ind]].reshape(96,96,1),M[ind],(cols,rows)).reshape(96,96,1) for ind in range(trans_train_data.shape[0])])
    
    x_col = [col for col in train_data.columns if ((col != 'Image') & ('_x' in col))]
    y_col = [col for col in train_data.columns if ((col != 'Image') & ('_y' in col))]

    shift_x_array = np.array([np.repeat(x,len(x_col))for x in shift_x])
    shift_y_array = np.array([np.repeat(y,len(y_col))for y in shift_y])
    
    trans_train_data[x_col] += shift_x_array
    trans_train_data[y_col] += shift_y_array
    
    trans_coord = np.array(trans_train_data[[col for col in trans_train_data.columns if col != 'Image']])

    # TODO should we force these to be nan or leave as is? If leave as is, comment this out.
    trans_coord[trans_coord > 96]= np.nan
    trans_coord[trans_coord < 0]= np.nan
    return(trans_img,trans_coord)

if add_translate_img:
    trans_img,trans_coord = translate_img()
    x_train = np.append(x_train,trans_img,axis=0)
    y_train = np.append(y_train,trans_coord,axis=0)


## Modeling

In [21]:
# Define callback function if detailed log required
class History(tensorflow.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_loss = []
        self.train_rmse = []
        self.val_rmse = []
        self.val_loss = []

    def on_batch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.train_rmse.append(logs.get('rmse'))
        
    def on_epoch_end(self, batch, logs={}):    
        self.val_rmse.append(logs.get('val_rmse'))
        self.val_loss.append(logs.get('val_loss'))
        
# Implement ModelCheckPoint callback function to save CNN model
class CNN_ModelCheckpoint(tensorflow.keras.callbacks.Callback):

    def __init__(self, model, filename):
        self.filename = filename
        self.cnn_model = model

    def on_train_begin(self, logs={}):
        self.max_val_rmse = math.inf
        
 
    def on_epoch_end(self, batch, logs={}):    
        val_rmse = logs.get('val_rmse')
        if(val_rmse < self.max_val_rmse):
           self.max_val_rmse = val_rmse
           self.cnn_model.save_weights(self.filename)


#### Primary Capsula Layer
The output is then reshaped into 8-dimensional vector. So shape will be 6x6x32 capsules each of which will be 8-dimensional. Then it will pass through a non-linear function(squash) so that length of output vector can be maintained between 0 and 1.

In [22]:
# convolution layer with stride 2 and 256 filters of size 9*9
#conv2 = Conv2D(256, (9,9), strides = 2, padding = 'valid')(conv1)
 
# reshape into 1152 capsules of 8 dimensional vectors
#reshaped = Reshape((6*6*32,8))(conv1)
 
def squash(inputs):
    # take norm of input vectors
    squared_norm = K.sum(K.square(inputs), axis = -1, keepdims = True)
 
    # use the formula for non-linear function to return squashed output
    return ((squared_norm/(1+squared_norm))/(K.sqrt(squared_norm+K.epsilon())))*inputs



#### Capsule Layer

Here we need to write a custom layer in keras. It will take 1152x8 as its input and produces output of size 30x16, where 30 capsules each represents an output class with 16 dimensional vector. Then each of these 30 capsules are converted into single value to predict the output class using a lambda layer.

In [26]:
class CapsuleLayer(Layer):
    # creating a layer class in keras
    def __init__(self, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.kernel_initializer = initializers.get('glorot_uniform')
    
    def build(self, input_shape): 
        # initialize weight matrix for each capsule in lower layer
        self.W = self.add_weight(shape = [30, 6*6*32, 16, 8], initializer = self.kernel_initializer, name = 'weights')
        self.built = True
    
    def call(self, inputs):
        inputs = K.expand_dims(inputs, 1)
        inputs = K.tile(inputs, [1, 30, 1, 1])
        # matrix multiplication b/w previous layer output and weight matrix
        inputs = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs)
        b = tf.zeros(shape = [K.shape(inputs)[0], 30, 6*6*32])
        
# routing algorithm with updating coupling coefficient c, using scalar product b/w input capsule and output capsule
        for i in range(3):
            c = tf.nn.softmax(b, dim=1)
            s = K.batch_dot(c, inputs, [2, 2])
            v = squash(s)
            b = b + K.batch_dot(v, inputs, [2,3])
            
        return v 
    def compute_output_shape(self, input_shape):
        return tuple([None, 30, 16])
    
    
    
def output_layer(inputs):
    return K.sqrt(K.sum(K.square(inputs), -1) + K.epsilon())
 


In [25]:
def capsule_model():
    input_shape = Input(shape=(96,96,1))  # size of input image is 96*96
 
# a convolution layer output shape = 20*20*256
#conv1 = Conv2D(256, (9,9), activation = 'relu', padding = 'valid')(input_shape)
    x = Conv2D(32, (3,3), padding='same', use_bias=False)(input_shape)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(96, (3,3), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3,3),padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(256, (3,3),padding='same',use_bias=False)(x)
    x = LeakyReLU(alpha = 0.1)(x)
    x = BatchNormalization()(x)
    
    #reshape into 1152 capsules of 8 dimensional vectors
    reshaped = Reshape((6*6*32,8))(x)
    
    # squash the reshaped output to make length of vector b/w 0 and 1
    squashed_output = Lambda(squash)(reshaped)
    
    caps_out = CapsuleLayer()(squashed_output)
    outputs = Lambda(output_layer)(caps_out)
    
    model = Model(input_shape, outputs, name="capsule_model")
    return model

model = capsule_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 96, 32)        288       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 48, 64)        18432     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 48, 48, 64)        0         
__________

In [28]:
# Custom RMSE metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

#from tensorflow.keras.optimizers.schedules import InverseTimeDecay

# Use Nadam optimizer with variable learning rate
optimizer = Nadam(lr=0.00001,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)


# Loss: MSE and Metric = RMSE
model.compile(optimizer= optimizer, 
              loss='mean_squared_error',
              metrics=[rmse])

#Callback to save the best model
saveBase_Model = CNN_ModelCheckpoint(model, model_dir+"capsule_model_weights.h5")

#define callback functions
callbacks = [#EarlyStopping(monitor='val_rmse', patience=3, verbose=2),
             saveBase_Model]

Run for 1000 epochs and keeping 20% train-valid split

In [29]:

USE_SAVED_MODEL = False

if USE_SAVED_MODEL == False:
    history = model.fit(x_train,
                    y_train,
                    epochs = 1000,
                    batch_size = 256,
                    validation_split = 0.2, #data = (x_test, y_test),
                    callbacks = callbacks
                    )
    plt.style.use('seaborn-darkgrid')
    plt.plot(history.history['rmse'])
    plt.xlabel('Epochs')
    plt.ylabel('Root Mean Square Error')
    plt.title('Model Training Error')
    plt.show() 
    
else:
    model.load_weights(model_dir+"capsule_model_weights.h5")
                    

Instructions for updating:
Use tf.cast instead.
Train on 7329 samples, validate on 1833 samples
Epoch 1/1000
4864/7329 [==================>...........] - ETA: 1:11 - loss: 2676.8676 - rmse: 51.7113

KeyboardInterrupt: 

In [ ]:
lookid_dir = '../input/IdLookupTable.csv'
lookid_data = pd.read_csv(lookid_dir)
test_data = pd.read_csv(test_file)

x_test = []
for i in range(0,len(test_data)):
    img = test_data['Image'][i].split(' ')
    x_test.append(img)
    
x_test = np.array(x_test,dtype = 'float')
x_test = x_test/255.0
x_test = x_test.reshape(-1,96,96,1)    

y_test = model.predict(x_test)
y_test = np.clip(y_test,0,96)

lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pred_list = list(y_test)

rowid = list(lookid_data['RowId'])

feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
    
    
submit_data = []
for x,y in zip(imageID,feature):
    submit_data.append(pred_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(submit_data,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('../output/w207_capsule_submission.csv',index = False)   


In [ ]:
if 1:
    sess = get_session()
    clear_session()
    sess.close()

    try:
        del model # this is from global space - change this as you need
    except:
        print("Model clear Failed")
    print(gc.collect())    